# 빅데이터 분석기사 실기 작업 1유형 연습문제

In [1]:
# 라이브러리 호출
import pandas as pd
import numpy as np

In [2]:
# 31. 서울시 따릉이 이용정보 데이터
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력
# 데이터 불러오기
csvFile = 'https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv'
df = pd.read_csv(csvFile)
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678950 entries, 0 to 678949
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   대여일자    678950 non-null  object 
 1   대여시간    678950 non-null  int64  
 2   대여소번호   678950 non-null  int64  
 3   대여구분코드  678950 non-null  object 
 4   성별      678950 non-null  object 
 5   연령대코드   678950 non-null  object 
 6   이용건수    678950 non-null  int64  
 7   운동량     678950 non-null  object 
 8   탄소량     678950 non-null  object 
 9   이동거리    678950 non-null  float64
 10  사용시간    678950 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 57.0+ MB


In [6]:
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력
result = df['대여일자'].value_counts().sort_index().to_frame()
ans = result[result['대여일자'] == result['대여일자'].max()].index[0]
display(result)
print(ans)

,대여일자
2021-06-01,48081
2021-06-02,56100
2021-06-03,18985
2021-06-04,56788
2021-06-05,52730
2021-06-06,47787
2021-06-07,52623
2021-06-08,55623
2021-06-09,54507
2021-06-10,36992


2021-06-04


In [7]:
# 32. 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력
# 데이터타입 변환
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()
result = df['day_name'].value_counts().to_frame()
print(result)

           day_name
Wednesday    110607
Friday       108877
Monday       107568
Tuesday      103704
Saturday     101299
Sunday        90918
Thursday      55977


In [8]:
# 33. 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력
result = df.groupby(['day_name', '대여소번호']).size().to_frame('size').sort_values(['day_name', 'size'], ascending = False).reset_index()
ans = result.drop_duplicates('day_name', keep = 'first').reset_index(drop = True)
display(ans)

,day_name,대여소번호,size
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


In [9]:
# 34. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라.
# 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함
daily = df[df['대여구분코드'].isin(['일일권', '일일권(비회원)'])]['연령대코드'].value_counts().sort_index()
total = df['연령대코드'].value_counts().sort_index()
ratio = (daily/total).sort_values(ascending = False)
print(ratio)
print('max ratio age', ratio.index[0])

~10대    0.312525
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
70대~    0.048962
60대     0.042462
Name: 연령대코드, dtype: float64
max ratio age ~10대


In [10]:
# 35. 연령대별 평균 이동거리
result = df[['연령대코드', '이동거리']].groupby(['연령대코드']).mean()
print(result)

              이동거리
연령대코드             
20대    3211.890552
30대    3341.443859
40대    3514.857416
50대    3593.668100
60대    3538.145737
70대~   3085.039641
~10대   2634.426279


In [11]:
# 36. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출
# 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하기
tw = df[df['연령대코드'] == '20대'].reset_index(drop = True)
# 데이터의 이동거리값의 평균 이상인 데이터 추출
tw_mean = tw[tw['이동거리'] >= tw['이동거리'].mean()].reset_index(drop = True)
# 데이터타입 변환
tw_mean['탄소량'] = tw_mean['탄소량'].astype('float')
target = tw_mean.sort_values(['대여일자', '대여소번호'], ascending = False).reset_index(drop = True).iloc[:200]['탄소량']
result = round(target.sum() / len(target), 3)
print(result)

1.613


In [12]:
# 37. 6월 7일 ~10대의 “이용건수”의 중앙값
# 데이터타입 변환
df['대여일자'] = pd.to_datetime(df['대여일자'])
result = df[(df['연령대코드'] == '~10대') & (df['대여일자'] == pd.to_datetime('2021-06-07'))]['이용건수'].median()
print(result)

1.0


In [13]:
# 38. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후
# 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력
target = df[(df['day_name'].isin(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Monday'])) & (df['대여시간'].isin([6, 7, 8]))]
result = target.groupby(['대여시간', '대여소번호']).size().to_frame('이용 횟수')

ans = result.sort_values(['대여시간', '이용 횟수'], ascending = False).groupby('대여시간').head(3)
display(ans)

이용 횟수
대여시간 대여소번호       
8    2701     119
     646      115
     1152      92
7    259      104
     230       77
     726       77
6    2744      45
     1125      40
     1028      36

In [14]:
# 39. 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값 구하기
result = df[df['이동거리'] >= df['이동거리'].mean()].reset_index(drop = True)['이동거리'].std()
print(result)

5092.139707505305


In [15]:
# 40. 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값
df['sex'] = df['성별'].map(lambda x: '남' if x in ['M', 'm'] else '여')
ans = df[['sex', '이동거리']].groupby('sex').mean()
display(ans)

,이동거리
sex,
남,3209.110871
여,3468.575025
